In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMinMax
import matplotlib.pyplot as plt
import joblib
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Bidirectional
from keras.regularizers import l2

In [2]:
file_path= "C:/Users/vgarciario/desktop/UA MASTER/labo3/final_dataset_descr.csv"
df = pd.read_csv(file_path, sep='\t')

In [3]:
# Paso 1: Filtrar los datos por los periodos 201907, 201908 y 201909
df_filtered = df[df['periodo'].isin(['201907', '201908', '201909'])]

# Pivotear los datos para tener columnas separadas para cada periodo
pivoted_sales = df_filtered.pivot_table(index=['product_id', 'customer_id'], columns='periodo', values='tn').reset_index()

# Asegurar que las columnas 201907 y 201909 existen en el DataFrame
pivoted_sales = pivoted_sales.reindex(columns=['product_id', 'customer_id', '201907', '201908', '201909'])

# Calcular el promedio de julio y septiembre para agosto
pivoted_sales['201908'] = pivoted_sales[['201907', '201909']].mean(axis=1)

# Convertir de nuevo al formato largo
updated_sales = pivoted_sales.melt(id_vars=['product_id', 'customer_id'], value_vars=['201907', '201908', '201909'], 
                                   var_name='periodo', value_name='tn')

# Unir con el dataframe original
df = df.drop(columns=['tn'])
df = pd.merge(df, updated_sales, on=['product_id', 'customer_id', 'periodo'], how='left')

In [4]:

# Paso 2: Buscar los 'product_id' que tengan poca historia
training_threshold = 3
product_history = df.groupby(['product_id', 'periodo']).size().reset_index(name='counts')
products_to_keep = product_history[product_history['counts'] >= training_threshold]['product_id'].unique()
df_filtered = df[df['product_id'].isin(products_to_keep)]

# Crear el DataFrame "Predicciones" para productos con poca historia
products_to_predict = product_history[product_history['counts'] < training_threshold]['product_id'].unique()
predicciones = df[df['product_id'].isin(products_to_predict)].groupby('product_id')['tn'].mean().reset_index()
predicciones.rename(columns={'tn': 'prediccion'}, inplace=True)

In [ ]:

# Paso 3: Aplicar LabelEncoder a las columnas categoricas
categorical_cols = ['cat1', 'cat2', 'cat3', 'brand', 'descripcion']
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df_filtered[col] = le.fit_transform(df_filtered[col])
    label_encoders[col] = le



In [7]:
# Paso 4: Agrupar las ventas por periodo, cat1, cat2, cat3, marca y descripcion
# Filtrar datos para el año 2019 (si 'periodo' es numérico)
df_2019 = df[(df['periodo'] >= 201901) & (df['periodo'] <= 201912)]

# Agrupar las ventas del año 2019 por periodo, cat1, cat2, cat3, brand, descripcion y product_id
grouped_sales_2019 = df_2019.groupby(['periodo', 'cat1', 'cat2', 'cat3', 'brand', 'descripcion', 'product_id'])['tn'].sum().reset_index()

# Calcular el total de ventas por grupo para el año 2019
group_totals_2019 = grouped_sales_2019.groupby(['cat1', 'cat2', 'cat3', 'brand', 'descripcion'])['tn'].sum().reset_index()

# Calcular el promedio de ventas por periodo para el año 2019
average_sales_2019 = grouped_sales_2019.groupby(['cat1', 'cat2', 'cat3', 'brand', 'descripcion', 'product_id'])['tn'].mean().reset_index()

# Unir para calcular el ratio
ratios_2019 = pd.merge(average_sales_2019, group_totals_2019, on=['cat1', 'cat2', 'cat3', 'brand', 'descripcion'], suffixes=('', '_total'))
ratios_2019['ratio'] = ratios_2019['tn'] / ratios_2019['tn_total']

# Crear un diccionario de ratios
ratio_dict = ratios_2019.set_index(['cat1', 'cat2', 'cat3', 'brand', 'descripcion', 'product_id'])['ratio'].to_dict()


In [10]:

# Paso 5: Agrupar las ventas por periodo, cat1, cat2, cat3, marca y customer_id
grouped_df = df_filtered.groupby(['periodo', 'cat1', 'cat2', 'cat3', 'brand', 'customer_id']).agg({
    'cust_request_qty': 'sum',
    'cust_request_tn': 'sum',
    'tn': 'sum'
}).reset_index()

# Crear un diccionario para almacenar los scalers
scalers = {}
scaled_df = grouped_df.copy()

# Aplicar StandardScaler a cada columna de interés
for col in ['cust_request_qty', 'cust_request_tn', 'tn']:
    scaler = StandardScaler()
    scaled_df[col] = scaler.fit_transform(scaled_df[[col]])
    scalers[col] = scaler

# Guardar los scalers para su uso posterior
joblib.dump(scalers, 'scalers.pkl')


['scalers.pkl']

In [11]:

# Paso 6: Agrupar en clusters con dtw y hacer grafico de codo para determinar la cantidad optima de clusters
pivot_table = scaled_df.pivot_table(index=['customer_id', 'cat1', 'cat2', 'cat3', 'brand'], columns='periodo', values='tn', fill_value=0)
time_series_data = pivot_table.values
scaler = TimeSeriesScalerMinMax()
time_series_data = scaler.fit_transform(time_series_data)

# Método del codo para determinar el número óptimo de clusters
distortions = []
K = range(1, 11)
for k in K:
    model = TimeSeriesKMeans(n_clusters=k, metric="dtw", max_iter=10, random_state=0)
    model.fit(time_series_data)
    distortions.append(model.inertia_)

# Graficar el codo
plt.figure(figsize=(10, 6))
plt.plot(K, distortions, 'bx-')
plt.xlabel('Número de clusters')
plt.ylabel('Distorsión (Inercia)')
plt.title('Método del Codo para determinar el número óptimo de clusters')
plt.show()

# Suponiendo que el codo sugiere 3 clusters (ajustar según el gráfico)
optimal_clusters = 3
model = TimeSeriesKMeans(n_clusters=optimal_clusters, metric="dtw", max_iter=10, random_state=0)
labels = model.fit_predict(time_series_data)
pivot_table['cluster'] = labels


KeyboardInterrupt: 

In [ ]:

# Paso 7: Armar un modelo LSTM bidireccional para predecir las ventas
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(Bidirectional(LSTM(128, activation='tanh', kernel_regularizer=l2(0.7), return_sequences=True), input_shape=input_shape))
    model.add(Dropout(0.1))
    model.add(Bidirectional(LSTM(256, activation='tanh', kernel_regularizer=l2(0.7), return_sequences=True)))
    model.add(Dropout(0.1))
    model.add(Bidirectional(LSTM(512, activation='tanh', kernel_regularizer=l2(0.7), return_sequences=True)))
    model.add(Dropout(0.1))
    model.add(Bidirectional(LSTM(256, activation='tanh', kernel_regularizer=l2(0.7), return_sequences=True)))
    model.add(Dropout(0.1))
    model.add(Bidirectional(LSTM(128, activation='relu', kernel_regularizer=l2(0.7))))
    model.add(Dropout(0.1))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

# Preparar datos y entrenar el modelo LSTM
grouped_df['periodo'] = pd.to_datetime(grouped_df['periodo'], format='%Y%m')
models = {}
predictions = []

for (cat1, cat2, cat3, brand, descripcion), group_data in grouped_df.groupby(['cat1', 'cat2', 'cat3', 'brand', 'descripcion']):
    group_data = group_data.sort_values(by='periodo')
    n_steps = 2
    X, y = [], []
    for i in range(len(group_data) - n_steps):
        X.append(group_data[['cust_request_qty', 'cust_request_tn', 'tn']].iloc[i:i+n_steps].values)
        y.append(group_data['tn'].iloc[i+n_steps])
    X, y = np.array(X), np.array(y)
    
    model = build_lstm_model((X.shape[1], X.shape[2]))
    model.fit(X, y, epochs=100, verbose=0)
    models[(cat1, cat2, cat3, brand, descripcion)] = model
    
    X_pred = group_data[['cust_request_qty', 'cust_request_tn', 'tn']].values[-n_steps:]
    X_pred = X_pred.reshape((1, X_pred.shape[0], X_pred.shape[1]))
    pred = model.predict(X_pred, verbose=0)
    predictions.append([cat1, cat2, cat3, brand, descripcion, pred[0][0]])

In [ ]:

# Paso 8: Convertir las predicciones a un DataFrame
pred_df = pd.DataFrame(predictions, columns=['cat1', 'cat2', 'cat3', 'brand', 'descripcion', 'prediccion'])

# Sumarizar las predicciones por grupo
summarized_preds = pred_df.groupby(['cat1', 'cat2', 'cat3', 'brand', 'descripcion'])['prediccion'].sum().reset_index()

In [ ]:

# Paso 9: Aplicar los ratios para obtener las predicciones finales por product_id
final_predictions = []
for _, row in summarized_preds.iterrows():
    key = (row['cat1'], row['cat2'], row['cat3'], row['brand'], row['descripcion'])
    for (cat1, cat2, cat3, brand, descripcion, product_id), ratio in ratio_dict.items():
        if (cat1, cat2, cat3, brand, descripcion) == key:
            final_predictions.append([product_id, row['prediccion'] * ratio])

# Convertir las predicciones finales a un DataFrame
final_predictions_df = pd.DataFrame(final_predictions, columns=['product_id', 'prediccion'])

# Paso 10: Desescalar las predicciones finales
scalers = joblib.load('scalers.pkl')
final_predictions_df['prediccion'] = scalers['tn'].inverse_transform(final_predictions_df[['prediccion']])

# Paso 11: Unificar con el DataFrame "Predicciones"
final_df = pd.concat([final_predictions_df, predicciones], ignore_index=True)

# Guardar el resultado en un archivo CSV
final_df.to_csv('predicciones_finales.csv', index=False)